# BEAST 2 Output Figures & Stats

The results in this notebook are from a phylodynamics pipeline using any BEAST 2 models.

In [ ]:
save_dir = None
beast_xml_path = None
xml_set_label = 'xml_set'

In [ ]:
import warnings
warnings.filterwarnings( "ignore", module = "matplotlib\..*" )
from beast_pype.outputs import read_xml_set_logs_for_plotting, plot_comparative_box_violin, plot_comparative_origin_or_tmrca, hdi_pivot
from beast_pype.date_utilities import date_to_decimal, decimal_to_date
import pandas as pd
from beast2xml import BEAST2XML
import os

In [ ]:
# This cell retrieves all the log files for the samples you selected.
merged_log_paths = {file.replace('_merged.log', ''):f'{save_dir}/{file}' for file in os.listdir(save_dir) if file.endswith('_merged.log')}
youngest_tip_year_decimals = {}
for xml_set, path in beast_xml_path.items():
    beast_xml = BEAST2XML(path)
    youngest_tip_year_decimals[xml_set] = beast_xml.extract_youngest_year_decimal()
df, df_melted_for_seaborn = read_xml_set_logs_for_plotting(
    file_path_dict=merged_log_paths,
    xml_set_label=xml_set_label,
    youngest_tip_dates_dict=youngest_tip_year_decimals,
    convert_become_uninfectious_rate=False)

# TMRCA

In [ ]:
if 'TMRCA' in df.columns:
    fig = plot_comparative_origin_or_tmrca(df_melted_for_seaborn, 'TMRCA', one_figure=True, xml_set_label=xml_set_label)

In [ ]:
if 'TMRCA' in df.columns:
    fig = plot_comparative_origin_or_tmrca(df_melted_for_seaborn, 'TMRCA', xml_set_label=xml_set_label)

In [ ]:
if 'TMRCA' in df.columns:
    tmrca_hdi_df = hdi_pivot(df, 'TMRCA', xml_set_label=xml_set_label)
    tmrca_hdi_df['Lower 0.95 HDI Date'] =  tmrca_hdi_df['Lower 0.95 HDI'].map(decimal_to_date).dt.strftime("%Y-%m-%dir")
    tmrca_hdi_df['Median Date'] =  tmrca_hdi_df['Median'].map(decimal_to_date).dt.strftime("%Y-%m-%dir")
    tmrca_hdi_df['Upper 0.95 HDI Date'] =  tmrca_hdi_df['Upper 0.95 HDI'].map(decimal_to_date).dt.strftime("%Y-%m-%dir")
    tmrca_hdi_df